# Title
[]()

In [2]:

import pandas as pd
import sys
import os
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\custom_python")
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\src")
from file_functions import *
import time
import re

from response_processing import *
from article_processing import create_text_dict_from_folder
import traceback
from file_functions import *

In [3]:
# set the option to wrap text within cells
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

# Set up

In [1]:
main_dict = dict()

#  *BMJ Open*: from `2023-06-16 web crawling` and `2023-06-18` notebooks

In [14]:
import scrapy
from scrapy.crawler import CrawlerRunner
from crochet import setup, wait_for
import re
from IPython import display
from pprint import pprint
setup()

def trim_text(text, regex=None):
    if regex==None:
        regex = '.*<h2>Abstract</h2>.*(?:Introduction.*)?(<h2.*?>Introduction</h2>.*References)<.*' 
    try:
        processed = re.search(regex, text, re.DOTALL).group(1)
        html_display = display.HTML(processed)
    except: 
        print('Unable to parse article text')
        processed = '<Error parsing article text>' 
        html_display = processed
    return processed, html_display

def text_dict_from_web(article_dict, header=2, to_display=0,
        regex_str='.*<h\d>Abstract</h\d>.*(?:Introduction.*)?(<h\d.*?>Introduction</h\d>.*References)<.*'
        ):
    """
    Create a text dictionary from a dictionary containing web-scraped articles.

    Parameters:
        article_dict (dict): Values of each dictionary item are a dictionary representing the data from a 
            single article: 'url', 'text', and 'title'.

    Returns:
        text_dict: Dictionary where each item is a string of the text of an article, starting with the title.
    """
    regex_str = regex_str.replace('\d', f'{header}')
    regex = rf'{regex_str}'
    print(f'Regex pattern: {regex}')
    text_dict = dict()
    display_dict = dict()
    if type(to_display) != list:
        to_display = [to_display] 
    for article_key in article_dict:
        trimmed_text, display = trim_text(article_dict[article_key]['text'], regex)
        text_dict[article_key] = f"{article_dict[article_key]['title']}\n\n{trimmed_text}"
        if article_key in to_display:
            display_dict[article_key] = display
    print(f'text_dict keys: {[key for key in text_dict.keys()]}')
    return text_dict, display_dict

class myspider(scrapy.Spider):
    name = "myspider"
    
    def start_requests(self):
        urls = ['https://emails.bmj.com/q/1fnLH65XUsNn7Iiph6kELOM/wv']
        for url in urls:
            yield scrapy.Request(url=url, callback=self.parse_front)
    
    def parse_front(self, response):
        self.article_title = response.css('a.art-title > font::text').extract()
        article_url = response.xpath('//a[@class="art-title"]/@href').extract()
        # self.article_title = [response.css('a.art-title > font::text').extract_first()]
        # article_url = [response.xpath('//a[@class="art-title"]/@href').extract_first()]
        for index, url in enumerate(article_url):
            article_dict[index] = dict()
            article_dict[index]['title'] = self.article_title[index]
            article_dict[index]['url'] = url
            yield response.follow(url=url, callback=self.parse_pages, cb_kwargs={'index': index})
    
    def parse_pages(self, response, index):
        text = response.xpath('//h2|//p|//h3|//h4').extract()
        article_dict[index]['text'] = ''.join([line for line in text])
        
@wait_for(10)
def run_spider():
    crawler = CrawlerRunner()
    d = crawler.crawl(myspider)
    return d



iteration_id = 1
# main_dict = dict()
article_dict = dict()

run_spider()
main_dict[iteration_id] = article_dict



In [22]:
text_dict, display_dict = text_dict_from_web(article_dict, to_display=[0])

Regex pattern: .*<h2>Abstract</h2>.*(?:Introduction.*)?(<h2.*?>Introduction</h2>.*References)<.*
Unable to parse article text
Unable to parse article text
text_dict keys: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]


## ## Sample parsed HTML from scraped article 

In [23]:
display_dict[0]

## Sample text from scraped article 

In [17]:
print(next(iter(article_dict.values()))['text'])

<p class="logo-bmj-journals"><a href="http://journals.bmj.com/" title="BMJ Journals"><img src="/sites/default/themes/bmjj/img/logos/logo-bmj-journals.svg" title="BMJ Journals" alt="BMJ Journals"></a></p><h3>Log in using your username and password</h3><h2 class="pane-title"><span class="icon-search"></span></h2><h2 class="pane-title"><span class="icon-menu"></span></h2><h2 class="pane-title">Main menu</h2><h3>Log in using your username and password</h3><h2 class="element-invisible">You are here</h2><p><span class="icon-article-text"></span>Article Text</p><p><a class="article-pdf-download" href="/content/bmjopen/13/6/e071489.full.pdf" target="new"><img alt="Download PDF" src="/sites/default/themes/bmjj/img/icon-pdf.png"><strong>PDF</strong></a></p><p><a class="article-pdf-download" href="/content/bmjopen/13/6/e071489.full.pdf?with-ds=yes" target="new"><img alt="Download PDF + Supplemental Data" src="/sites/default/themes/bmjj/img/icon-pdf.png"><strong>PDF +<br>
Supplementary<br>
Materia

## Article titles

In [21]:
for article in article_dict:
    print(article_dict[article]['title'])
    print('\n')

What does high value care for musculoskeletal conditions mean and how do you apply it in practice? A consensus statement from a research network of physiotherapists in New South Wales, Australia


Physical activity promotion in daycare centres in Germany: study protocol for a cross-sectional survey within the BeweKi study


National and international programmatic perspective on facilitators and barriers for Sudan’s health sector response on female genital mutilation (2016–2018): a qualitative study


Examination of the demographic representativeness of a cross-sectional mobile phone survey in collecting health data in Colombia using random digit dialling


Undernutrition among children living in refugee camps: a systematic review of prevalence


Incidence of child marriage among refugees and internally displaced persons in the Middle East and South Asia: evidence from six cross-sectional surveys


Towards Response ADAptive Radiotherapy for organ preservation for intermediate-risk recta

## Iteration 1.1: update regex based on journal title

In [24]:
import scrapy
from scrapy.crawler import CrawlerRunner
from crochet import setup, wait_for
import re
from IPython import display
import time
from pprint import pprint
setup()

def trim_text(text, regex=None):
    if regex==None:
        regex = '.*<h2>Abstract</h2>.*(?:Introduction.*)?(<h2.*?>Introduction</h2>.*References)<.*' 
    try:
        processed = re.search(regex, text, re.DOTALL).group(1)
        html_display = display.HTML(processed)
    except: 
        print('Unable to parse article text')
        processed = '<Error parsing article text>' 
        html_display = processed
    return processed, html_display

def text_dict_from_web(article_dict, header=2, to_display=0,
        regex_str='.*<h\d>Abstract</h\d>.*(?:Introduction.*)?(<h\d.*?>Introduction</h\d>.*References)<.*'
        ):
    """
    Create a text dictionary from a dictionary containing web-scraped articles.

    Parameters:
        article_dict (dict): Values of each dictionary item are a dictionary representing the data from a 
            single article: 'url', 'text', and 'title'.

    Returns:
        text_dict: Dictionary where each item is a string of the text of an article, starting with the title.
    """
    journal = article_dict[0]['journal']
    print(f'Parsing {len(article_dict)} articles from {journal}')
    regex_str = regex_str.replace('\d', f'{header}')
    regex = rf'{regex_str}'
    print(f'Regex pattern: {regex}')
    text_dict = dict()
    display_dict = dict()
    if type(to_display) != list:
        to_display = [to_display] 
    for article_key in article_dict:
        trimmed_text, display = trim_text(article_dict[article_key]['text'], regex)
        text_dict[article_key] = f"{article_dict[article_key]['title']}\n\n{trimmed_text}"
        if article_key in to_display:
            display_dict[article_key] = display
    print(f'text_dict keys: {[key for key in text_dict.keys()]}')
    return text_dict, display_dict

class myspider(scrapy.Spider):
    name = "myspider"
    
    def start_requests(self):
        urls = ['https://emails.bmj.com/q/1fnLH65XUsNn7Iiph6kELOM/wv']
        for url in urls:
            yield scrapy.Request(url=url, callback=self.parse_front)
    
    def parse_front(self, response):
        # self.article_title = response.css('a.art-title > font::text').extract()
        # article_url = response.xpath('//a[@class="art-title"]/@href').extract()
        self.article_title = [response.css('a.art-title > font::text').extract_first()]
        article_url = [response.xpath('//a[@class="art-title"]/@href').extract_first()]
        for index, url in enumerate(article_url):
            article_dict[index] = dict()
            article_dict[index]['journal'] = 'BMJ Open'
            article_dict[index]['title'] = self.article_title[index]
            article_dict[index]['url'] = url
            yield response.follow(url=url, callback=self.parse_pages, cb_kwargs={'index': index})
    
    def parse_pages(self, response, index):
        text = response.xpath('//h2|//p|//h3|//h4').extract()
        article_dict[index]['text'] = ''.join([line for line in text])
        
@wait_for(10)
def run_spider():
    crawler = CrawlerRunner()
    d = crawler.crawl(myspider)
    return d



iteration_id = 1.1
# main_dict = dict()
article_dict = dict()

run_spider()
main_dict[iteration_id] = article_dict

time.sleep(10)
text_dict, display_dict = text_dict_from_web(article_dict, to_display=[0])

Parsing 1 articles from BMJ Open
Regex pattern: .*<h2>Abstract</h2>.*(?:Introduction.*)?(<h2.*?>Introduction</h2>.*References)<.*
text_dict keys: [0]


In [25]:
display_dict[0]

# *PLOS One*

## unable to scrape from RSS

In [6]:
import scrapy
from scrapy.crawler import CrawlerRunner
from crochet import setup, wait_for
import re
from IPython import display
import time
from pprint import pprint
setup()

def trim_text(text, regex=None):
    if regex==None:
        regex = '.*<h2>Abstract</h2>.*(?:Introduction.*)?(<h2.*?>Introduction</h2>.*References)<.*' 
    try:
        processed = re.search(regex, text, re.DOTALL).group(1)
        html_display = display.HTML(processed)
    except: 
        print('Unable to parse article text')
        processed = '<Error parsing article text>' 
        html_display = processed
    return processed, html_display

def text_dict_from_web(article_dict, header=2, to_display=0,
        regex_str='.*<h\d>Abstract</h\d>.*(?:Introduction.*)?(<h\d.*?>Introduction</h\d>.*References)<.*'
        ):
    """
    Create a text dictionary from a dictionary containing web-scraped articles.

    Parameters:
        article_dict (dict): Values of each dictionary item are a dictionary representing the data from a 
            single article: 'url', 'text', and 'title'.

    Returns:
        text_dict: Dictionary where each item is a string of the text of an article, starting with the title.
    """
    journal = article_dict[0]['journal']
    print(f'Parsing {len(article_dict)} articles from {journal}')
    regex_str = regex_str.replace('\d', f'{header}')
    regex = rf'{regex_str}'
    print(f'Regex pattern: {regex}')
    text_dict = dict()
    display_dict = dict()
    if type(to_display) != list:
        to_display = [to_display] 
    for article_key in article_dict:
        trimmed_text, display = trim_text(article_dict[article_key]['text'], regex)
        text_dict[article_key] = f"{article_dict[article_key]['title']}\n\n{trimmed_text}"
        if article_key in to_display:
            display_dict[article_key] = display
    print(f'text_dict keys: {[key for key in text_dict.keys()]}')
    return text_dict, display_dict

class crawler_RSS1(scrapy.Spider):
    name = "crawler_RSS1"
    
    def start_requests(self):
        urls = ['https://journals.plos.org/plosone/feed/atom']
        for url in urls:
            yield scrapy.Request(url=url, callback=self.parse_front)
    
    def parse_front(self, response):
        article_url = response.xpath('/text()').extract()
        # self.article_title = response.xpath('//entry/title/text()').extract()
        # article_url = response.xpath('//entry/link[@rel="alternate"]/@href').extract()
        # article_url = response.css('entry > link[rel="alternate"][href]').extract()
        # self.article_title = response.css('entry > link[rel="alternate"]').extract()
        # article_url = response.css('entry > link[rel="alternate"][href]').extract()
        # self.article_title = response.css('link[rel="alternate"][title]').extract()
        # article_url = response.css('link[rel="alternate"][href]').extract()


        # article_url = response.xpath('entry > link::attr()"]/@href').extract()
        # self.article_title = [response.css('a.art-title > font::text').extract_first()]
        # article_url = [response.xpath('//a[@class="art-title"]/@href').extract_first()]
        for index, url in enumerate(article_url):
            print(url)
            # article_dict[index] = dict()
            # article_dict[index]['journal'] = 'BMJ Open'
            # article_dict[index]['title'] = self.article_title[index]
            # article_dict[index]['url'] = url
    #         yield response.follow(url=url, callback=self.parse_pages, cb_kwargs={'index': index})
    
    # def parse_pages(self, response, index):
    #     text = response.xpath('//h2|//p|//h3|//h4').extract()
    #     article_dict[index]['text'] = ''.join([line for line in text])
        
@wait_for(10)
def run_spider():
    crawler = CrawlerRunner()
    d = crawler.crawl(crawler_RSS1)
    return d



iteration_id = 2
# main_dict = dict()
article_dict = dict()

run_spider()
main_dict[iteration_id] = article_dict
article_dict
# time.sleep(10)
# text_dict, display_dict = text_dict_from_web(article_dict, to_display=[0])

{}

## 2.1 able to scrape from web page URL

In [12]:
import scrapy
from scrapy.crawler import CrawlerRunner
from crochet import setup, wait_for
import re
from IPython import display
import time
from pprint import pprint
setup()

def trim_text(text, regex=None):
    if regex==None:
        regex = '.*<h2>Abstract</h2>.*(?:Introduction.*)?(<h2.*?>Introduction</h2>.*References)<.*' 
    try:
        processed = re.search(regex, text, re.DOTALL).group(1)
        html_display = display.HTML(processed)
    except: 
        print('Unable to parse article text')
        processed = '<Error parsing article text>' 
        html_display = processed
    return processed, html_display

def text_dict_from_web(article_dict, header=2, to_display=0,
        regex_str='.*<h\d>Abstract</h\d>.*(?:Introduction.*)?(<h\d.*?>Introduction</h\d>.*References)<.*'
        ):
    """
    Create a text dictionary from a dictionary containing web-scraped articles.

    Parameters:
        article_dict (dict): Values of each dictionary item are a dictionary representing the data from a 
            single article: 'url', 'text', and 'title'.

    Returns:
        text_dict: Dictionary where each item is a string of the text of an article, starting with the title.
    """
    journal = article_dict[0]['journal']
    print(f'Parsing {len(article_dict)} articles from {journal}')
    regex_str = regex_str.replace('\d', f'{header}')
    regex = rf'{regex_str}'
    print(f'Regex pattern: {regex}')
    text_dict = dict()
    display_dict = dict()
    if type(to_display) != list:
        to_display = [to_display] 
    for article_key in article_dict:
        trimmed_text, display = trim_text(article_dict[article_key]['text'], regex)
        text_dict[article_key] = f"{article_dict[article_key]['title']}\n\n{trimmed_text}"
        if article_key in to_display:
            display_dict[article_key] = display
    print(f'text_dict keys: {[key for key in text_dict.keys()]}')
    return text_dict, display_dict

class crawler_RSS1(scrapy.Spider):
    name = "crawler_RSS1"
    
    def start_requests(self):
        urls = ['https://journals.plos.org/plosone/browse/']
        for url in urls:
            yield scrapy.Request(url=url, callback=self.parse_front)
    
    def parse_front(self, response):
        article_url = response.xpath('//div[@class="article-block"]/a[@class="article-url"]/@href').extract()
        # self.article_title = response.xpath('//entry/title/text()').extract()
        # article_url = response.xpath('//entry/link[@rel="alternate"]/@href').extract()
        # article_url = response.css('entry > link[rel="alternate"][href]').extract()
        # self.article_title = response.css('entry > link[rel="alternate"]').extract()
        # article_url = response.css('entry > link[rel="alternate"][href]').extract()
        # self.article_title = response.css('link[rel="alternate"][title]').extract()
        # article_url = response.css('link[rel="alternate"][href]').extract()


        # article_url = response.xpath('entry > link::attr()"]/@href').extract()
        # self.article_title = [response.css('a.art-title > font::text').extract_first()]
        # article_url = [response.xpath('//a[@class="art-title"]/@href').extract_first()]
        for index, url in enumerate(article_url):
            print(url)
            # article_dict[index] = dict()
            # article_dict[index]['journal'] = 'BMJ Open'
            # article_dict[index]['title'] = self.article_title[index]
            # article_dict[index]['url'] = url
    #         yield response.follow(url=url, callback=self.parse_pages, cb_kwargs={'index': index})
    
    # def parse_pages(self, response, index):
    #     text = response.xpath('//h2|//p|//h3|//h4').extract()
    #     article_dict[index]['text'] = ''.join([line for line in text])
        
@wait_for(10)
def run_spider():
    crawler = CrawlerRunner()
    d = crawler.crawl(crawler_RSS1)
    return d



iteration_id = 2
# main_dict = dict()
article_dict = dict()

run_spider()
main_dict[iteration_id] = article_dict
article_dict
# time.sleep(10)
# text_dict, display_dict = text_dict_from_web(article_dict, to_display=[0])

hello /plosone/article?id=10.1371/journal.pone.0287468
hello /plosone/article?id=10.1371/journal.pone.0287463
hello /plosone/article?id=10.1371/journal.pone.0287250
hello /plosone/article?id=10.1371/journal.pone.0287135
hello /plosone/article?id=10.1371/journal.pone.0286993
hello /plosone/article?id=10.1371/journal.pone.0286970
hello /plosone/article?id=10.1371/journal.pone.0286829
hello /plosone/article?id=10.1371/journal.pone.0286759
hello /plosone/article?id=10.1371/journal.pone.0284567
hello /plosone/article?id=10.1371/journal.pone.0283754
hello /plosone/article?id=10.1371/journal.pone.0287475
hello /plosone/article?id=10.1371/journal.pone.0287447
hello /plosone/article?id=10.1371/journal.pone.0287433


{}

## 2.2 works with web page; need to modify the script to work with XML

In [25]:
import scrapy
from scrapy.crawler import CrawlerRunner
from crochet import setup, wait_for
import re
from IPython import display
import time
from pprint import pprint
setup()

def trim_text(text, regex=None):
    if regex==None:
        regex = '.*<h2>Abstract</h2>.*(?:Introduction.*)?(<h2.*?>Introduction</h2>.*References)<.*' 
    try:
        processed = re.search(regex, text, re.DOTALL).group(1)
        html_display = display.HTML(processed)
    except: 
        print('Unable to parse article text')
        processed = '<Error parsing article text>' 
        html_display = processed
    return processed, html_display

def text_dict_from_web(article_dict, header=2, to_display=0,
        regex_str='.*<h\d>Abstract</h\d>.*(?:Introduction.*)?(<h\d.*?>Introduction</h\d>.*References)<.*'
        ):
    """
    Create a text dictionary from a dictionary containing web-scraped articles.

    Parameters:
        article_dict (dict): Values of each dictionary item are a dictionary representing the data from a 
            single article: 'url', 'text', and 'title'.

    Returns:
        text_dict: Dictionary where each item is a string of the text of an article, starting with the title.
    """
    journal = article_dict[0]['journal']
    print(f'Parsing {len(article_dict)} articles from {journal}')
    regex_str = regex_str.replace('\d', f'{header}')
    regex = rf'{regex_str}'
    print(f'Regex pattern: {regex}')
    text_dict = dict()
    display_dict = dict()
    if type(to_display) != list:
        to_display = [to_display] 
    for article_key in article_dict:
        trimmed_text, display = trim_text(article_dict[article_key]['text'], regex)
        text_dict[article_key] = f"{article_dict[article_key]['title']}\n\n{trimmed_text}"
        if article_key in to_display:
            display_dict[article_key] = display
    print(f'text_dict keys: {[key for key in text_dict.keys()]}')
    return text_dict, display_dict

class crawler_RSS1(scrapy.Spider):
    name = "crawler_RSS1"
    
    def start_requests(self):
        urls = ['https://journals.plos.org/plosone/browse/']
        for url in urls:
            yield scrapy.Request(url=url, callback=self.parse_front)
    
    def parse_front(self, response):
        article_url = [response.xpath('//div[@class="article-block"]/a[@class="article-url"]/@href').get()]
        # self.article_title = response.xpath('//entry/title/text()').extract()
        # article_url = response.xpath('//entry/link[@rel="alternate"]/@href').extract()
        # article_url = response.css('entry > link[rel="alternate"][href]').extract()
        # self.article_title = response.css('entry > link[rel="alternate"]').extract()
        # article_url = response.css('entry > link[rel="alternate"][href]').extract()
        # self.article_title = response.css('link[rel="alternate"][title]').extract()
        # article_url = response.css('link[rel="alternate"][href]').extract()


        # article_url = response.xpath('entry > link::attr()"]/@href').extract()
        # self.article_title = [response.css('a.art-title > font::text').extract_first()]
        # article_url = [response.xpath('//a[@class="art-title"]/@href').extract_first()]
        for index, url in enumerate(article_url):
            print(url)
            # article_dict[index] = dict()
            # article_dict[index]['journal'] = 'BMJ Open'
            # article_dict[index]['title'] = self.article_title[index]
            # article_dict[index]['url'] = url
    #         yield response.follow(url=url, callback=self.parse_pages, cb_kwargs={'index': index})
    
    # def parse_pages(self, response, index):
    #     text = response.xpath('//h2|//p|//h3|//h4').extract()
    #     article_dict[index]['text'] = ''.join([line for line in text])
        
@wait_for(10)
def run_spider():
    crawler = CrawlerRunner()
    d = crawler.crawl(crawler_RSS1)
    return d



iteration_id = 2
# main_dict = dict()
article_dict = dict()

run_spider()
main_dict[iteration_id] = article_dict
article_dict
# time.sleep(10)
# text_dict, display_dict = text_dict_from_web(article_dict, to_display=[0])

/plosone/article?id=10.1371/journal.pone.0287468


{}

## 2.3

In [26]:
import scrapy
from scrapy.crawler import CrawlerRunner
from crochet import setup, wait_for
import re
from IPython import display
import time
from scrapy.http import XmlResponse
from pprint import pprint
setup()

def trim_text(text, regex=None):
    if regex==None:
        regex = '.*<h2>Abstract</h2>.*(?:Introduction.*)?(<h2.*?>Introduction</h2>.*References)<.*' 
    try:
        processed = re.search(regex, text, re.DOTALL).group(1)
        html_display = display.HTML(processed)
    except: 
        print('Unable to parse article text')
        processed = '<Error parsing article text>' 
        html_display = processed
    return processed, html_display

def text_dict_from_web(article_dict, header=2, to_display=0,
        regex_str='.*<h\d>Abstract</h\d>.*(?:Introduction.*)?(<h\d.*?>Introduction</h\d>.*References)<.*'
        ):
    """
    Create a text dictionary from a dictionary containing web-scraped articles.

    Parameters:
        article_dict (dict): Values of each dictionary item are a dictionary representing the data from a 
            single article: 'url', 'text', and 'title'.

    Returns:
        text_dict: Dictionary where each item is a string of the text of an article, starting with the title.
    """
    journal = article_dict[0]['journal']
    print(f'Parsing {len(article_dict)} articles from {journal}')
    regex_str = regex_str.replace('\d', f'{header}')
    regex = rf'{regex_str}'
    print(f'Regex pattern: {regex}')
    text_dict = dict()
    display_dict = dict()
    if type(to_display) != list:
        to_display = [to_display] 
    for article_key in article_dict:
        trimmed_text, display = trim_text(article_dict[article_key]['text'], regex)
        text_dict[article_key] = f"{article_dict[article_key]['title']}\n\n{trimmed_text}"
        if article_key in to_display:
            display_dict[article_key] = display
    print(f'text_dict keys: {[key for key in text_dict.keys()]}')
    return text_dict, display_dict

class crawler_RSS1(scrapy.Spider):
    name = "crawler_RSS1"
    
    def start_requests(self):
        urls = ['https://journals.plos.org/plosone/feed/atom']
        for url in urls:
            yield XmlResponse(url=url, callback=self.parse_front)
    
    def parse_front(self, response):
        article_url = [response.xpath('//title').get()]
        # self.article_title = response.xpath('//entry/title/text()').extract()
        # article_url = response.xpath('//entry/link[@rel="alternate"]/@href').extract()
        # article_url = response.css('entry > link[rel="alternate"][href]').extract()
        # self.article_title = response.css('entry > link[rel="alternate"]').extract()
        # article_url = response.css('entry > link[rel="alternate"][href]').extract()
        # self.article_title = response.css('link[rel="alternate"][title]').extract()
        # article_url = response.css('link[rel="alternate"][href]').extract()


        # article_url = response.xpath('entry > link::attr()"]/@href').extract()
        # self.article_title = [response.css('a.art-title > font::text').extract_first()]
        # article_url = [response.xpath('//a[@class="art-title"]/@href').extract_first()]
        for index, url in enumerate(article_url):
            print(url)
            # article_dict[index] = dict()
            # article_dict[index]['journal'] = 'BMJ Open'
            # article_dict[index]['title'] = self.article_title[index]
            # article_dict[index]['url'] = url
    #         yield response.follow(url=url, callback=self.parse_pages, cb_kwargs={'index': index})
    
    # def parse_pages(self, response, index):
    #     text = response.xpath('//h2|//p|//h3|//h4').extract()
    #     article_dict[index]['text'] = ''.join([line for line in text])
        
@wait_for(10)
def run_spider():
    crawler = CrawlerRunner()
    d = crawler.crawl(crawler_RSS1)
    return d



iteration_id = 2.3
# main_dict = dict()
article_dict = dict()

run_spider()
main_dict[iteration_id] = article_dict
article_dict
# time.sleep(10)
# text_dict, display_dict = text_dict_from_web(article_dict, to_display=[0])

{}

## 2.4

In [30]:
import scrapy
from scrapy.crawler import CrawlerRunner
from crochet import setup, wait_for
import re
from IPython import display
import time
from pprint import pprint
setup()

def trim_text(text, regex=None):
    if regex==None:
        regex = '.*<h2>Abstract</h2>.*(?:Introduction.*)?(<h2.*?>Introduction</h2>.*References)<.*' 
    try:
        processed = re.search(regex, text, re.DOTALL).group(1)
        html_display = display.HTML(processed)
    except: 
        print('Unable to parse article text')
        processed = '<Error parsing article text>' 
        html_display = processed
    return processed, html_display

def text_dict_from_web(article_dict, header=2, to_display=0,
        regex_str='.*<h\d>Abstract</h\d>.*(?:Introduction.*)?(<h\d.*?>Introduction</h\d>.*References)<.*'
        ):
    """
    Create a text dictionary from a dictionary containing web-scraped articles.

    Parameters:
        article_dict (dict): Values of each dictionary item are a dictionary representing the data from a 
            single article: 'url', 'text', and 'title'.

    Returns:
        text_dict: Dictionary where each item is a string of the text of an article, starting with the title.
    """
    journal = article_dict[0]['journal']
    print(f'Parsing {len(article_dict)} articles from {journal}')
    regex_str = regex_str.replace('\d', f'{header}')
    regex = rf'{regex_str}'
    print(f'Regex pattern: {regex}')
    text_dict = dict()
    display_dict = dict()
    if type(to_display) != list:
        to_display = [to_display] 
    for article_key in article_dict:
        trimmed_text, display = trim_text(article_dict[article_key]['text'], regex)
        text_dict[article_key] = f"{article_dict[article_key]['title']}\n\n{trimmed_text}"
        if article_key in to_display:
            display_dict[article_key] = display
    print(f'text_dict keys: {[key for key in text_dict.keys()]}')
    return text_dict, display_dict

class crawler_RSS1(scrapy.Spider):
    name = "crawler_RSS1"
    
    def start_requests(self):
        urls = ['https://journals.plos.org/plosone/browse/']
        for url in urls:
            yield scrapy.Request(url=url, callback=self.parse_front)
    
    def parse_front(self, response):
        article_url = [response.xpath('//text()').getall()]
        # self.article_title = response.xpath('//entry/title/text()').extract()
        # article_url = response.xpath('//entry/link[@rel="alternate"]/@href').extract()
        # article_url = response.css('entry > link[rel="alternate"][href]').extract()
        # self.article_title = response.css('entry > link[rel="alternate"]').extract()
        # article_url = response.css('entry > link[rel="alternate"][href]').extract()
        # self.article_title = response.css('link[rel="alternate"][title]').extract()
        # article_url = response.css('link[rel="alternate"][href]').extract()


        # article_url = response.xpath('entry > link::attr()"]/@href').extract()
        # self.article_title = [response.css('a.art-title > font::text').extract_first()]
        # article_url = [response.xpath('//a[@class="art-title"]/@href').extract_first()]
        for index, url in enumerate(article_url):
            print(url)
            # article_dict[index] = dict()
            # article_dict[index]['journal'] = 'BMJ Open'
            # article_dict[index]['title'] = self.article_title[index]
            # article_dict[index]['url'] = url
    #         yield response.follow(url=url, callback=self.parse_pages, cb_kwargs={'index': index})
    
    # def parse_pages(self, response, index):
    #     text = response.xpath('//h2|//p|//h3|//h4').extract()
    #     article_dict[index]['text'] = ''.join([line for line in text])
        
@wait_for(10)
def run_spider():
    crawler = CrawlerRunner()
    d = crawler.crawl(crawler_RSS1)
    return d



iteration_id = 2.4
# main_dict = dict()
article_dict = dict()

run_spider()
main_dict[iteration_id] = article_dict
article_dict
# time.sleep(10)
# text_dict, display_dict = text_dict_from_web(article_dict, to_display=[0])

['\n\n\n\n', '\n\n\n\n\n', '\n  ', '\n\n\n', '\n', '\n\n', '\n    ', '\n        var siteUrlPrefix = "/plosone/";\n    ', '\n  ', '\n  ', '\n\n  ', '\n  ', '\n\n\n\n\n\n', '\n', "\n  var googletag = googletag || {};\n  googletag.cmd = googletag.cmd || [];\n  (function() {\n    var gads = document.createElement('script');\n    gads.async = true;\n    gads.type = 'text/javascript';\n    var useSSL = 'https:' == document.location.protocol;\n    gads.src = (useSSL ? 'https:' : 'http:') +\n        '//www.googletagservices.com/tag/js/gpt.js';\n    var node = document.getElementsByTagName('script')[0];\n    node.parentNode.insertBefore(gads, node);\n  })();\n", '\n\n', '\n\n  ', "\n    googletag.cmd.push(function() {\n  googletag.defineSlot('/75507958/PONE_728x90_ATF', [728, 90], 'div-gpt-ad-1458247671871-0').addService(googletag.pubads());\n  googletag.defineSlot('/75507958/PONE_160x600_BTF', [160, 600], 'div-gpt-ad-1458247671871-1').addService(googletag.pubads());\n  googletag.defineSlot('/7

{}

## 2.5 

In [42]:

urls = {
    'PLOS One': 'https://journals.plos.org/plosone/feed/atom'
    }
for url in urls:
    print(url)

PLOS One


In [49]:
import scrapy
from scrapy.crawler import CrawlerRunner
from crochet import setup, wait_for
import re
from IPython import display
import time
from pprint import pprint
setup()

def trim_text(text, regex=None):
    if regex==None:
        regex = '.*<h2>Abstract</h2>.*(?:Introduction.*)?(<h2.*?>Introduction</h2>.*References)<.*' 
    try:
        processed = re.search(regex, text, re.DOTALL).group(1)
        html_display = display.HTML(processed)
    except: 
        print('Unable to parse article text')
        processed = '<Error parsing article text>' 
        html_display = processed
    return processed, html_display

def text_dict_from_web(article_dict, header=2, to_display=0,
        regex_str='.*<h\d>Abstract</h\d>.*(?:Introduction.*)?(<h\d.*?>Introduction</h\d>.*References)<.*'
        ):
    """
    Create a text dictionary from a dictionary containing web-scraped articles.

    Parameters:
        article_dict (dict): Values of each dictionary item are a dictionary representing the data from a 
            single article: 'url', 'text', and 'title'.

    Returns:
        text_dict: Dictionary where each item is a string of the text of an article, starting with the title.
    """
    journal = article_dict[0]['journal']
    print(f'Parsing {len(article_dict)} articles from {journal}')
    regex_str = regex_str.replace('\d', f'{header}')
    regex = rf'{regex_str}'
    print(f'Regex pattern: {regex}')
    text_dict = dict()
    display_dict = dict()
    if type(to_display) != list:
        to_display = [to_display] 
    for article_key in article_dict:
        trimmed_text, display = trim_text(article_dict[article_key]['text'], regex)
        text_dict[article_key] = f"{article_dict[article_key]['title']}\n\n{trimmed_text}"
        if article_key in to_display:
            display_dict[article_key] = display
    print(f'text_dict keys: {[key for key in text_dict.keys()]}')
    return text_dict, display_dict

class crawler_RSS1(scrapy.Spider):
    name = "crawler_RSS1"
    
    def start_requests(self):
        journals = {
            'PLOS One': 'https://journals.plos.org/plosone/feed/atom'
            }
        for journal in journals:
            yield scrapy.Request(url=journals[journal], callback=self.parse_front, cb_kwargs={'journal': journal})
    
    def parse_front(self, response, journal):
        response.selector.remove_namespaces() # This is needed for any Atom feeds
        # self.article_title = response.xpath('//entry/title/text()').getall()
        # article_url = response.css('entry > link[rel="alternate"]::attr(href)').getall()
        self.article_title = [response.xpath('//entry/title/text()').get()]
        article_url = [response.css('entry > link[rel="alternate"]::attr(href)').get()]
        for index, url in enumerate(article_url):
            article_dict[index] = dict()
            article_dict[index]['journal'] = journal
            article_dict[index]['title'] = self.article_title[index]
            article_dict[index]['url'] = url
            yield response.follow(url=url, callback=self.parse_pages, cb_kwargs={'index': index})
    
    def parse_pages(self, response, index):
        text = response.xpath('//h2|//p|//h3|//h4').extract()
        article_dict[index]['text'] = ''.join([line for line in text])
        
@wait_for(10)
def run_spider():
    crawler = CrawlerRunner()
    d = crawler.crawl(crawler_RSS1)
    return d



iteration_id = 2.5
# main_dict = dict()
article_dict = dict()

run_spider()
main_dict[iteration_id] = article_dict
article_dict
# time.sleep(10)
# text_dict, display_dict = text_dict_from_web(article_dict, to_display=[0])

{0: {'journal': 'PLOS One',
  'title': 'Perceived impacts of COVID-19 and bushfires on the implementation of an obesity prevention trial in Northeast Victoria, Australia',
  'url': 'https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0287468',
  'text': '<h3 class="callout-headline">Submit Your Manuscript</h3><p class="callout-content">\n  Discover a faster, simpler path to publishing in a high-quality journal. <em>PLOS ONE</em> promises fair, rigorous peer review,\n  broad scope, and wide readership – a perfect fit for your research every time.\n  </p><p class="button-contain special">\n    <a class="button button-default" href="/plosone/static/publish">\n     Learn More\n    </a>\n    <a class="button-link" href="https://www.editorialmanager.com/pone/default.asp">\n      Submit Now\n    </a>\n  </p><p>Click through the PLOS taxonomy to find articles in your field.</p><p>For more information about PLOS Subject Areas, click\n          <a href="https://github.com/PLOS/plos

## 2.51

In [63]:
import scrapy
from scrapy.crawler import CrawlerRunner
from crochet import setup, wait_for
import re
from IPython import display
import time
from pprint import pprint
setup()

def trim_text(text, regex=None):
    if regex==None:
        regex = '.*<h2>Abstract</h2>.*(?:Introduction.*)?(<h2.*?>Introduction</h2>.*References)<.*' 
    try:
        processed = re.search(regex, text, re.DOTALL).group(1)
        html_display = display.HTML(processed)
    except: 
        print('Unable to parse article text')
        processed = '<Error parsing article text>' 
        html_display = processed
    return processed, html_display

def text_dict_from_web(article_dict, header=2, to_display=0,
        regex_str='.*<h\d>Abstract</h\d>.*(?:Introduction.*)?(<h\d.*?>Introduction</h\d>.*References)<.*'
        ):
    """
    Create a text dictionary from a dictionary containing web-scraped articles.

    Parameters:
        article_dict (dict): Values of each dictionary item are a dictionary representing the data from a 
            single article: 'url', 'text', and 'title'.

    Returns:
        text_dict: Dictionary where each item is a string of the text of an article, starting with the title.
    """
    journal = article_dict[0]['journal']
    print(f'Parsing {len(article_dict)} articles from {journal}')
    regex_str = regex_str.replace('\d', f'{header}')
    regex = rf'{regex_str}'
    print(f'Regex pattern: {regex}')
    text_dict = dict()
    display_dict = dict()
    if type(to_display) != list:
        to_display = [to_display] 
    for article_key in article_dict:
        trimmed_text, display = trim_text(article_dict[article_key]['text'], regex)
        text_dict[article_key] = f"{article_dict[article_key]['title']}\n\n{trimmed_text}"
        if article_key in to_display:
            display_dict[article_key] = display
    print(f'text_dict keys: {[key for key in text_dict.keys()]}')
    return text_dict, display_dict

class crawler_RSS1(scrapy.Spider):
    name = "crawler_RSS1"
    
    def start_requests(self):
        journals = {
            'PLOS One': 'https://journals.plos.org/plosone/feed/atom'
            }
        for journal in journals:
            yield scrapy.Request(url=journals[journal], callback=self.parse_front, cb_kwargs={'journal': journal})
    
    def parse_front(self, response, journal):
        response.selector.remove_namespaces() # This is needed for any Atom feeds
        self.article_title = response.xpath('//entry/title/text()').getall()
        article_url = response.css('entry > link[rel="alternate"]::attr(href)').getall()
        # self.article_title = [response.xpath('//entry/title/text()').get()]
        # article_url = [response.css('entry > link[rel="alternate"]::attr(href)').get()]
        for index, url in enumerate(article_url):
            article_dict[index] = dict()
            article_dict[index]['journal'] = journal
            article_dict[index]['title'] = self.article_title[index]
            article_dict[index]['url'] = url
            yield response.follow(url=url, callback=self.parse_pages, cb_kwargs={'index': index})
    
    def parse_pages(self, response, index):
        text = response.xpath('//h2|//p|//h3|//h4').extract()
        article_dict[index]['text'] = ''.join([line for line in text])
        
@wait_for(10)
def run_spider():
    crawler = CrawlerRunner()
    d = crawler.crawl(crawler_RSS1)
    return d



iteration_id = 2.51
# main_dict = dict()
article_dict = dict()

run_spider()
main_dict[iteration_id] = article_dict
article_dict.keys()
# time.sleep(10)
# text_dict, display_dict = text_dict_from_web(article_dict, to_display=[0])

dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29])

In [140]:
next(iter(main_dict[2.51].values()))

{'journal': 'PLOS One',
 'title': 'Perceived impacts of COVID-19 and bushfires on the implementation of an obesity prevention trial in Northeast Victoria, Australia',
 'url': 'https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0287468',
 'text': '<h3 class="callout-headline">Submit Your Manuscript</h3><p class="callout-content">\n  Discover a faster, simpler path to publishing in a high-quality journal. <em>PLOS ONE</em> promises fair, rigorous peer review,\n  broad scope, and wide readership – a perfect fit for your research every time.\n  </p><p class="button-contain special">\n    <a class="button button-default" href="/plosone/static/publish">\n     Learn More\n    </a>\n    <a class="button-link" href="https://www.editorialmanager.com/pone/default.asp">\n      Submit Now\n    </a>\n  </p><p>Click through the PLOS taxonomy to find articles in your field.</p><p>For more information about PLOS Subject Areas, click\n          <a href="https://github.com/PLOS/plos-thesau

In [144]:
for article in main_dict[2.51]:
    print(main_dict[2.51][article])

{'journal': 'PLOS One', 'title': 'Perceived impacts of COVID-19 and bushfires on the implementation of an obesity prevention trial in Northeast Victoria, Australia', 'url': 'https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0287468', 'text': '<h3 class="callout-headline">Submit Your Manuscript</h3><p class="callout-content">\n  Discover a faster, simpler path to publishing in a high-quality journal. <em>PLOS ONE</em> promises fair, rigorous peer review,\n  broad scope, and wide readership – a perfect fit for your research every time.\n  </p><p class="button-contain special">\n    <a class="button button-default" href="/plosone/static/publish">\n     Learn More\n    </a>\n    <a class="button-link" href="https://www.editorialmanager.com/pone/default.asp">\n      Submit Now\n    </a>\n  </p><p>Click through the PLOS taxonomy to find articles in your field.</p><p>For more information about PLOS Subject Areas, click\n          <a href="https://github.com/PLOS/plos-thesaurus

In [145]:
main_dict[2.51][29]

{'journal': 'PLOS One',
 'title': 'Research on low-carbon dual channel supply chain considering product substitution under government carbon tax and low-carbon subsidy',
 'url': 'https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0287167',
 'text': '<h3 class="callout-headline">Submit Your Manuscript</h3><p class="callout-content">\n  Discover a faster, simpler path to publishing in a high-quality journal. <em>PLOS ONE</em> promises fair, rigorous peer review,\n  broad scope, and wide readership – a perfect fit for your research every time.\n  </p><p class="button-contain special">\n    <a class="button button-default" href="/plosone/static/publish">\n     Learn More\n    </a>\n    <a class="button-link" href="https://www.editorialmanager.com/pone/default.asp">\n      Submit Now\n    </a>\n  </p><p>Click through the PLOS taxonomy to find articles in your field.</p><p>For more information about PLOS Subject Areas, click\n          <a href="https://github.com/PLOS/plos-the

## 2.6 multiple journals with Atom RSS feeds

In [57]:
import scrapy
from scrapy.crawler import CrawlerRunner
from crochet import setup, wait_for
import re
from IPython import display
import time
from pprint import pprint
setup()

def trim_text(text, regex=None):
    if regex==None:
        regex = '.*<h2>Abstract</h2>.*(?:Introduction.*)?(<h2.*?>Introduction</h2>.*References)<.*' 
    try:
        processed = re.search(regex, text, re.DOTALL).group(1)
        html_display = display.HTML(processed)
    except: 
        print('Unable to parse article text')
        processed = '<Error parsing article text>' 
        html_display = processed
    return processed, html_display

def text_dict_from_web(article_dict, header=2, to_display=0,
        regex_str='.*<h\d>Abstract</h\d>.*(?:Introduction.*)?(<h\d.*?>Introduction</h\d>.*References)<.*'
        ):
    """
    Create a text dictionary from a dictionary containing web-scraped articles.

    Parameters:
        article_dict (dict): Values of each dictionary item are a dictionary representing the data from a 
            single article: 'url', 'text', and 'title'.

    Returns:
        text_dict: Dictionary where each item is a string of the text of an article, starting with the title.
    """
    journal = article_dict[0]['journal']
    print(f'Parsing {len(article_dict)} articles from {journal}')
    regex_str = regex_str.replace('\d', f'{header}')
    regex = rf'{regex_str}'
    print(f'Regex pattern: {regex}')
    text_dict = dict()
    display_dict = dict()
    if type(to_display) != list:
        to_display = [to_display] 
    for article_key in article_dict:
        trimmed_text, display = trim_text(article_dict[article_key]['text'], regex)
        text_dict[article_key] = f"{article_dict[article_key]['title']}\n\n{trimmed_text}"
        if article_key in to_display:
            display_dict[article_key] = display
    print(f'text_dict keys: {[key for key in text_dict.keys()]}')
    return text_dict, display_dict

class crawler_RSS1(scrapy.Spider):
    name = "crawler_RSS1"
    
    def start_requests(self):
        journals = {
            'PLOS One': 'https://journals.plos.org/plosone/feed/atom',
            'Annual Review of Medicine': 'https://www.annualreviews.org/action/showFeed?ui=45mu4&mi=3fndc3&ai=sm&jc=med&type=etoc&feed=atom'
            }
        for journal in journals:
            yield scrapy.Request(url=journals[journal], callback=self.parse_front, cb_kwargs={'journal': journal})
    
    def parse_front(self, response, journal):
        response.selector.remove_namespaces() # This is needed for any Atom feeds
        self.article_title = response.xpath('//entry/title/text()').getall()
        article_url = response.css('entry > link[rel="alternate"]::attr(href)').getall()
        # self.article_title = [response.xpath('//entry/title/text()').get()]
        # article_url = [response.css('entry > link[rel="alternate"]::attr(href)').get()]
        for index, url in enumerate(article_url):
            article_dict[index] = dict()
            article_dict[index]['journal'] = journal
            article_dict[index]['title'] = self.article_title[index]
            article_dict[index]['url'] = url
            yield response.follow(url=url, callback=self.parse_pages, cb_kwargs={'index': index})
    
    def parse_pages(self, response, index):
        text = response.xpath('//h2|//p|//h3|//h4').extract()
        article_dict[index]['text'] = ''.join([line for line in text])
        
@wait_for(10)
def run_spider():
    crawler = CrawlerRunner()
    d = crawler.crawl(crawler_RSS1)
    return d



iteration_id = 2.6
# main_dict = dict()
article_dict = dict()

run_spider()
main_dict[iteration_id] = article_dict
article_dict.keys()
# time.sleep(10)
# text_dict, display_dict = text_dict_from_web(article_dict, to_display=[0])

dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34])

In [59]:
for article in article_dict:
    print(article_dict[article]['journal'])

PLOS One
PLOS One
PLOS One
PLOS One
PLOS One
PLOS One
PLOS One
PLOS One
PLOS One
PLOS One
PLOS One
PLOS One
PLOS One
PLOS One
PLOS One
PLOS One
PLOS One
PLOS One
PLOS One
PLOS One
PLOS One
PLOS One
PLOS One
PLOS One
PLOS One
PLOS One
PLOS One
PLOS One
PLOS One
PLOS One
Annual Review of Medicine
Annual Review of Medicine
Annual Review of Medicine
Annual Review of Medicine
Annual Review of Medicine


## 2.61

In [60]:
import scrapy
from scrapy.crawler import CrawlerRunner
from crochet import setup, wait_for
import re
from IPython import display
import time
from pprint import pprint
setup()

def trim_text(text, regex=None):
    if regex==None:
        regex = '.*<h2>Abstract</h2>.*(?:Introduction.*)?(<h2.*?>Introduction</h2>.*References)<.*' 
    try:
        processed = re.search(regex, text, re.DOTALL).group(1)
        html_display = display.HTML(processed)
    except: 
        print('Unable to parse article text')
        processed = '<Error parsing article text>' 
        html_display = processed
    return processed, html_display

def text_dict_from_web(article_dict, header=2, to_display=0,
        regex_str='.*<h\d>Abstract</h\d>.*(?:Introduction.*)?(<h\d.*?>Introduction</h\d>.*References)<.*'
        ):
    """
    Create a text dictionary from a dictionary containing web-scraped articles.

    Parameters:
        article_dict (dict): Values of each dictionary item are a dictionary representing the data from a 
            single article: 'url', 'text', and 'title'.

    Returns:
        text_dict: Dictionary where each item is a string of the text of an article, starting with the title.
    """
    journal = article_dict[0]['journal']
    print(f'Parsing {len(article_dict)} articles from {journal}')
    regex_str = regex_str.replace('\d', f'{header}')
    regex = rf'{regex_str}'
    print(f'Regex pattern: {regex}')
    text_dict = dict()
    display_dict = dict()
    if type(to_display) != list:
        to_display = [to_display] 
    for article_key in article_dict:
        trimmed_text, display = trim_text(article_dict[article_key]['text'], regex)
        text_dict[article_key] = f"{article_dict[article_key]['title']}\n\n{trimmed_text}"
        if article_key in to_display:
            display_dict[article_key] = display
    print(f'text_dict keys: {[key for key in text_dict.keys()]}')
    return text_dict, display_dict

class crawler_RSS1(scrapy.Spider):
    name = "crawler_RSS1"
    
    def start_requests(self):
        journals = {
            # 'PLOS One': 'https://journals.plos.org/plosone/feed/atom',
            'Annual Review of Medicine': 'https://www.annualreviews.org/action/showFeed?ui=45mu4&mi=3fndc3&ai=sm&jc=med&type=etoc&feed=atom'
            }
        for journal in journals:
            yield scrapy.Request(url=journals[journal], callback=self.parse_front, cb_kwargs={'journal': journal})
    
    def parse_front(self, response, journal):
        response.selector.remove_namespaces() # This is needed for any Atom feeds
        self.article_title = response.xpath('//entry/title/text()').getall()
        article_url = response.css('entry > link[rel="alternate"]::attr(href)').getall()
        # self.article_title = [response.xpath('//entry/title/text()').get()]
        # article_url = [response.css('entry > link[rel="alternate"]::attr(href)').get()]
        for index, url in enumerate(article_url):
            article_dict[index] = dict()
            article_dict[index]['journal'] = journal
            article_dict[index]['title'] = self.article_title[index]
            article_dict[index]['url'] = url
            yield response.follow(url=url, callback=self.parse_pages, cb_kwargs={'index': index})
    
    def parse_pages(self, response, index):
        text = response.xpath('//h2|//p|//h3|//h4').extract()
        article_dict[index]['text'] = ''.join([line for line in text])
        
@wait_for(10)
def run_spider():
    crawler = CrawlerRunner()
    d = crawler.crawl(crawler_RSS1)
    return d



iteration_id = 2.61
# main_dict = dict()
article_dict = dict()

run_spider()
main_dict[iteration_id] = article_dict
article_dict.keys()
# time.sleep(10)
# text_dict, display_dict = text_dict_from_web(article_dict, to_display=[0])

dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34])

In [61]:
for article in article_dict:
    print(article_dict[article]['journal'])

Annual Review of Medicine
Annual Review of Medicine
Annual Review of Medicine
Annual Review of Medicine
Annual Review of Medicine
Annual Review of Medicine
Annual Review of Medicine
Annual Review of Medicine
Annual Review of Medicine
Annual Review of Medicine
Annual Review of Medicine
Annual Review of Medicine
Annual Review of Medicine
Annual Review of Medicine
Annual Review of Medicine
Annual Review of Medicine
Annual Review of Medicine
Annual Review of Medicine
Annual Review of Medicine
Annual Review of Medicine
Annual Review of Medicine
Annual Review of Medicine
Annual Review of Medicine
Annual Review of Medicine
Annual Review of Medicine
Annual Review of Medicine
Annual Review of Medicine
Annual Review of Medicine
Annual Review of Medicine
Annual Review of Medicine
Annual Review of Medicine
Annual Review of Medicine
Annual Review of Medicine
Annual Review of Medicine
Annual Review of Medicine


## 2.62

In [76]:
import scrapy
from scrapy.crawler import CrawlerRunner
from crochet import setup, wait_for
import re
from IPython import display
import time
from pprint import pprint
setup()

def trim_text(text, regex=None):
    if regex==None:
        regex = '.*<h2>Abstract</h2>.*(?:Introduction.*)?(<h2.*?>Introduction</h2>.*References)<.*' 
    try:
        processed = re.search(regex, text, re.DOTALL).group(1)
        html_display = display.HTML(processed)
    except: 
        print('Unable to parse article text')
        processed = '<Error parsing article text>' 
        html_display = processed
    return processed, html_display

def text_dict_from_web(article_dict, header=2, to_display=0,
        regex_str='.*<h\d>Abstract</h\d>.*(?:Introduction.*)?(<h\d.*?>Introduction</h\d>.*References)<.*'
        ):
    """
    Create a text dictionary from a dictionary containing web-scraped articles.

    Parameters:
        article_dict (dict): Values of each dictionary item are a dictionary representing the data from a 
            single article: 'url', 'text', and 'title'.

    Returns:
        text_dict: Dictionary where each item is a string of the text of an article, starting with the title.
    """
    journal = article_dict[0]['journal']
    print(f'Parsing {len(article_dict)} articles from {journal}')
    regex_str = regex_str.replace('\d', f'{header}')
    regex = rf'{regex_str}'
    print(f'Regex pattern: {regex}')
    text_dict = dict()
    display_dict = dict()
    if type(to_display) != list:
        to_display = [to_display] 
    for article_key in article_dict:
        trimmed_text, display = trim_text(article_dict[article_key]['text'], regex)
        text_dict[article_key] = f"{article_dict[article_key]['title']}\n\n{trimmed_text}"
        if article_key in to_display:
            display_dict[article_key] = display
    print(f'text_dict keys: {[key for key in text_dict.keys()]}')
    return text_dict, display_dict

class crawler_RSS1(scrapy.Spider):
    name = "crawler_RSS1"
    
    def start_requests(self):
        journals = {
            'PLOS One': 'https://journals.plos.org/plosone/feed/atom',
            'Annual Review of Medicine': 'https://www.annualreviews.org/action/showFeed?ui=45mu4&mi=3fndc3&ai=sm&jc=med&type=etoc&feed=atom'
            }
        index = 0
        for journal in journals:
            yield scrapy.Request(url=journals[journal], callback=self.parse_front, cb_kwargs={'journal': journal, 'index': index})
    
    def parse_front(self, response, journal, index):
        response.selector.remove_namespaces() # This is needed for any Atom feeds
        self.article_title = response.xpath('//entry/title/text()').getall()
        article_url = response.css('entry > link[rel="alternate"]::attr(href)').getall()
        # self.article_title = [response.xpath('//entry/title/text()').get()]
        # article_url = [response.css('entry > link[rel="alternate"]::attr(href)').get()]
        index += 1
        for url in article_url:
            article_dict[index] = dict()
            article_dict[index]['journal'] = journal
            article_dict[index]['title'] = self.article_title[index]
            article_dict[index]['url'] = url
            print(index, journal)
            index += 1
            yield response.follow(url=url, callback=self.parse_pages, cb_kwargs={'index': index})
    
    def parse_pages(self, response, index):
        text = response.xpath('//h2|//p|//h3|//h4').extract()
        article_dict[index]['text'] = ''.join([line for line in text])
        
@wait_for(10)
def run_spider():
    crawler = CrawlerRunner()
    d = crawler.crawl(crawler_RSS1)
    return d



iteration_id = 2.61
# main_dict = dict()
article_dict = dict()

run_spider()
main_dict[iteration_id] = article_dict
article_dict.keys()
# time.sleep(10)
# text_dict, display_dict = text_dict_from_web(article_dict, to_display=[0])

1 Annual Review of Medicine
2 Annual Review of Medicine
3 Annual Review of Medicine
4 Annual Review of Medicine
5 Annual Review of Medicine
6 Annual Review of Medicine
7 Annual Review of Medicine
8 Annual Review of Medicine
9 Annual Review of Medicine
10 Annual Review of Medicine
11 Annual Review of Medicine
12 Annual Review of Medicine
13 Annual Review of Medicine
14 Annual Review of Medicine
15 Annual Review of Medicine
16 Annual Review of Medicine
17 Annual Review of Medicine
18 Annual Review of Medicine
19 Annual Review of Medicine
20 Annual Review of Medicine
21 Annual Review of Medicine
22 Annual Review of Medicine
23 Annual Review of Medicine
24 Annual Review of Medicine
25 Annual Review of Medicine
26 Annual Review of Medicine
27 Annual Review of Medicine
28 Annual Review of Medicine
29 Annual Review of Medicine
30 Annual Review of Medicine
31 Annual Review of Medicine
32 Annual Review of Medicine
33 Annual Review of Medicine
34 Annual Review of Medicine
1 PLOS One
2 PLOS One
3

dict_keys([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35])

In [75]:
for article in article_dict:
    print(article_dict[article]['journal'])

PLOS One
PLOS One
PLOS One
PLOS One
PLOS One
PLOS One
PLOS One
PLOS One
PLOS One
PLOS One
PLOS One
PLOS One
PLOS One
PLOS One
PLOS One
PLOS One
PLOS One
PLOS One
PLOS One
PLOS One
PLOS One
PLOS One
PLOS One
PLOS One
PLOS One
PLOS One
PLOS One
PLOS One
PLOS One
PLOS One
Annual Review of Medicine
Annual Review of Medicine
Annual Review of Medicine
Annual Review of Medicine
Annual Review of Medicine


## 2.63

In [132]:
import scrapy
from scrapy.crawler import CrawlerRunner
from crochet import setup, wait_for
import re
from IPython import display
import time
from pprint import pprint
setup()

def trim_text(text, regex=None):
    if regex==None:
        regex = '.*<h2>Abstract</h2>.*(?:Introduction.*)?(<h2.*?>Introduction</h2>.*References)<.*' 
    try:
        processed = re.search(regex, text, re.DOTALL).group(1)
        html_display = display.HTML(processed)
    except: 
        print('Unable to parse article text')
        processed = '<Error parsing article text>' 
        html_display = processed
    return processed, html_display

def text_dict_from_web(article_dict, header=2, to_display=0,
        regex_str='.*<h\d>Abstract</h\d>.*(?:Introduction.*)?(<h\d.*?>Introduction</h\d>.*References)<.*'
        ):
    """
    Create a text dictionary from a dictionary containing web-scraped articles.

    Parameters:
        article_dict (dict): Values of each dictionary item are a dictionary representing the data from a 
            single article: 'url', 'text', and 'title'.

    Returns:
        text_dict: Dictionary where each item is a string of the text of an article, starting with the title.
    """
    journal = article_dict[0]['journal']
    print(f'Parsing {len(article_dict)} articles from {journal}')
    regex_str = regex_str.replace('\d', f'{header}')
    regex = rf'{regex_str}'
    print(f'Regex pattern: {regex}')
    text_dict = dict()
    display_dict = dict()
    if type(to_display) != list:
        to_display = [to_display] 
    for article_key in article_dict:
        trimmed_text, display = trim_text(article_dict[article_key]['text'], regex)
        text_dict[article_key] = f"{article_dict[article_key]['title']}\n\n{trimmed_text}"
        if article_key in to_display:
            display_dict[article_key] = display
    print(f'text_dict keys: {[key for key in text_dict.keys()]}')
    return text_dict, display_dict

class crawler_RSS1(scrapy.Spider):
    name = "crawler_RSS1"
    
    def __init__(self):
        self.article_dict = {}  # Initialize article_dict dictionary

    def start_requests(self):
        journals = {
            'PLOS One': 'https://journals.plos.org/plosone/feed/atom',
            'Annual Review of Medicine': 'https://www.annualreviews.org/action/showFeed?ui=45mu4&mi=3fndc3&ai=sm&jc=med&type=etoc&feed=atom'
            }
        for journal_index, journal in enumerate(journals):
            yield scrapy.Request(url=journals[journal], callback=self.parse_front, cb_kwargs={'journal': journal, 'journal_index': journal_index})
    
    def parse_front(self, response, journal, journal_index):
        response.selector.remove_namespaces() # This is needed for any Atom feeds
        self.article_title = response.xpath('//entry/title/text()').getall()
        article_url = response.css('entry > link[rel="alternate"]::attr(href)').getall()
        # self.article_title = [response.xpath('//entry/title/text()').get()]
        # article_url = [response.css('entry > link[rel="alternate"]::attr(href)').get()]
        for index, url in enumerate(article_url):
            article_dict[f'{journal_index}.{index}'] = dict()
            article_dict[f'{journal_index}.{index}']['journal'] = journal
            article_dict[f'{journal_index}.{index}']['title'] = self.article_title[index]
            article_dict[f'{journal_index}.{index}']['url'] = url
            # print(index, journal)
            yield response.follow(url=url, callback=self.parse_pages2, cb_kwargs={'index': index, 'journal_index': journal_index})
    
    def parse_pages2(self, response, index, journal_index):
        print('Extracting text for article', f'{journal_index}.{index}')
        text = response.xpath('//h2|//p|//h3|//h4').extract()
        # print(text)
        article_dict[f'{journal_index}.{index}']['text'] = ''.join([line+'\n' for line in text])
        # print(article_dict[f'{journal_index}.{index}']['text'])
        yield article_dict
        
        
@wait_for(10)
def run_spider():
    crawler = CrawlerRunner()
    d = crawler.crawl(crawler_RSS1)
    return d



iteration_id = 2.63
# main_dict = dict()
# article_dict = dict()

run_spider()
main_dict[iteration_id] = article_dict
article_dict.keys()
# time.sleep(10)
# text_dict, display_dict = text_dict_from_web(article_dict, to_display=[0])

Extracting text for article 0.21
Extracting text for article 0.28
Extracting text for article 0.24
Extracting text for article 0.26
Extracting text for article 0.25
Extracting text for article 0.23
Extracting text for article 0.20
Extracting text for article 0.22
Extracting text for article 0.29
Extracting text for article 0.19
Extracting text for article 0.16
Extracting text for article 0.18
Extracting text for article 0.17
Extracting text for article 0.27
Extracting text for article 0.11
Extracting text for article 0.15
Extracting text for article 0.10
Extracting text for article 0.13
Extracting text for article 0.12
Extracting text for article 0.9
Extracting text for article 0.8
Extracting text for article 0.7
Extracting text for article 0.5
Extracting text for article 0.0
Extracting text for article 0.14
Extracting text for article 0.6
Extracting text for article 0.3
Extracting text for article 0.4
Extracting text for article 0.2
Extracting text for article 0.1


dict_keys(['1.0', '1.1', '1.2', '1.3', '1.4', '1.5', '1.6', '1.7', '1.8', '1.9', '1.10', '1.11', '1.12', '1.13', '1.14', '1.15', '1.16', '1.17', '1.18', '1.19', '1.20', '1.21', '1.22', '1.23', '1.24', '1.25', '1.26', '1.27', '1.28', '1.29', '1.30', '1.31', '1.32', '1.33', '1.34', '0.0', '0.1', '0.2', '0.3', '0.4', '0.5', '0.6', '0.7', '0.8', '0.9', '0.10', '0.11', '0.12', '0.13', '0.14', '0.15', '0.16', '0.17', '0.18', '0.19', '0.20', '0.21', '0.22', '0.23', '0.24', '0.25', '0.26', '0.27', '0.28', '0.29'])

In [134]:
for article in article_dict:
    print(f"{article_dict[article]['journal']}: {article_dict[article]['title']}")
    # print(f"{article}: {article_dict[article].keys()}")

Annual Review of Medicine: COVID-19 and Kidney Disease
Annual Review of Medicine: COVID-19 Thrombotic Complications and Therapeutic Strategies
Annual Review of Medicine: COVID-19: Challenges of Viral Variants
Annual Review of Medicine: Post-COVID-19 Condition
Annual Review of Medicine: SARS-CoV-2 Vaccination-Induced Thrombotic Thrombocytopenia: A Rare but Serious Immunologic Complication
Annual Review of Medicine: Endocrine Disorders and COVID-19
Annual Review of Medicine: Cytomegalovirus Therapy: Role of Letermovir in Prophylaxis and Treatment in Transplant Recipients
Annual Review of Medicine: Gender-Affirming Care of Transgender and Gender-Diverse Youth: Current Concepts
Annual Review of Medicine: Update in Adult Transgender Medicine
Annual Review of Medicine: New Frontiers in Obesity Treatment: GLP-1 and Nascent Nutrient-Stimulated Hormone-Based Therapeutics
Annual Review of Medicine: Advances and Applications of Polygenic Scores for Coronary Artery Disease
Annual Review of Medicin

In [133]:
for article in article_dict:
    print(f"{article_dict[article]['journal']}: {article_dict[article]['text'][:100]}")

KeyError: 'text'

In [131]:
(next(iter(main_dict[iteration_id].values())))

{'journal': 'Annual Review of Medicine',
 'title': 'COVID-19 and Kidney Disease',
 'url': 'https://www.annualreviews.org/doi/abs/10.1146/annurev-med-042420-104753?ai=sm&mi=3fndc3&af=A'}

In [151]:
main_dict[2.63].keys()

dict_keys(['1.0', '1.1', '1.2', '1.3', '1.4', '1.5', '1.6', '1.7', '1.8', '1.9', '1.10', '1.11', '1.12', '1.13', '1.14', '1.15', '1.16', '1.17', '1.18', '1.19', '1.20', '1.21', '1.22', '1.23', '1.24', '1.25', '1.26', '1.27', '1.28', '1.29', '1.30', '1.31', '1.32', '1.33', '1.34', '0.0', '0.1', '0.2', '0.3', '0.4', '0.5', '0.6', '0.7', '0.8', '0.9', '0.10', '0.11', '0.12', '0.13', '0.14', '0.15', '0.16', '0.17', '0.18', '0.19', '0.20', '0.21', '0.22', '0.23', '0.24', '0.25', '0.26', '0.27', '0.28', '0.29'])

In [153]:
main_dict[2.63]['1.4']

{'journal': 'Annual Review of Medicine',
 'title': 'SARS-CoV-2 Vaccination-Induced Thrombotic Thrombocytopenia: A Rare but Serious Immunologic Complication',
 'url': 'https://www.annualreviews.org/doi/abs/10.1146/annurev-med-043021-015237?ai=sm&mi=3fndc3&af=A'}

## 2.64 use yield item

In [141]:
import scrapy
from scrapy.crawler import CrawlerRunner
from crochet import setup, wait_for
import re
from IPython import display
import time
from pprint import pprint
setup()

def trim_text(text, regex=None):
    if regex==None:
        regex = '.*<h2>Abstract</h2>.*(?:Introduction.*)?(<h2.*?>Introduction</h2>.*References)<.*' 
    try:
        processed = re.search(regex, text, re.DOTALL).group(1)
        html_display = display.HTML(processed)
    except: 
        print('Unable to parse article text')
        processed = '<Error parsing article text>' 
        html_display = processed
    return processed, html_display

def text_dict_from_web(article_dict, header=2, to_display=0,
        regex_str='.*<h\d>Abstract</h\d>.*(?:Introduction.*)?(<h\d.*?>Introduction</h\d>.*References)<.*'
        ):
    """
    Create a text dictionary from a dictionary containing web-scraped articles.

    Parameters:
        article_dict (dict): Values of each dictionary item are a dictionary representing the data from a 
            single article: 'url', 'text', and 'title'.

    Returns:
        text_dict: Dictionary where each item is a string of the text of an article, starting with the title.
    """
    journal = article_dict[0]['journal']
    print(f'Parsing {len(article_dict)} articles from {journal}')
    regex_str = regex_str.replace('\d', f'{header}')
    regex = rf'{regex_str}'
    print(f'Regex pattern: {regex}')
    text_dict = dict()
    display_dict = dict()
    if type(to_display) != list:
        to_display = [to_display] 
    for article_key in article_dict:
        trimmed_text, display = trim_text(article_dict[article_key]['text'], regex)
        text_dict[article_key] = f"{article_dict[article_key]['title']}\n\n{trimmed_text}"
        if article_key in to_display:
            display_dict[article_key] = display
    print(f'text_dict keys: {[key for key in text_dict.keys()]}')
    return text_dict, display_dict

class crawler_RSS1(scrapy.Spider):
    name = "crawler_RSS1"
    
    # def __init__(self):
    #     self.article_dict = {}  # Initialize article_dict dictionary

    def start_requests(self):
        journals = {
            'PLOS One': 'https://journals.plos.org/plosone/feed/atom',
            'Annual Review of Medicine': 'https://www.annualreviews.org/action/showFeed?ui=45mu4&mi=3fndc3&ai=sm&jc=med&type=etoc&feed=atom'
            }
        for journal_index, journal in enumerate(journals):
            yield scrapy.Request(
                url=journals[journal], callback=self.parse_front, cb_kwargs={'journal': journal, 'journal_index': journal_index})
    
    def parse_front(self, response, journal, journal_index):
        response.selector.remove_namespaces() # This is needed for any Atom feeds
        # self.article_title = response.xpath('//entry/title/text()').getall()
        # article_url = response.css('entry > link[rel="alternate"]::attr(href)').getall()
        self.article_title = [response.xpath('//entry/title/text()').get()]
        article_url = [response.css('entry > link[rel="alternate"]::attr(href)').get()]
        for index, url in enumerate(article_url):
            article_dict[f'{journal_index}.{index}'] = dict()
            article_dict[f'{journal_index}.{index}']['journal'] = journal
            article_dict[f'{journal_index}.{index}']['title'] = self.article_title[index]
            article_dict[f'{journal_index}.{index}']['url'] = url
            article_item = article_dict[f'{journal_index}.{index}']
            yield response.follow(url=url, callback=self.parse_pages3, cb_kwargs={'article_item': article_item})
    
    def parse_pages3(self, response, article_item):
        text = response.xpath('//h2|//p|//h3|//h4').extract()
        # print(text)
        article_item['text'] = ''.join([line+'\n' for line in text])
        article_item['testing'] = 'testing'
        print(article_item['text'])
        # yield article_item
        
        
@wait_for(10)
def run_spider():
    crawler = CrawlerRunner()
    d = crawler.crawl(crawler_RSS1)
    return d



iteration_id = 2.64
# main_dict = dict()
article_dict = dict()

run_spider()
main_dict[iteration_id] = article_dict
article_dict.keys()
# time.sleep(10)
# text_dict, display_dict = text_dict_from_web(article_dict, to_display=[0])

<h3 class="callout-headline">Submit Your Manuscript</h3>
<p class="callout-content">
  Discover a faster, simpler path to publishing in a high-quality journal. <em>PLOS ONE</em> promises fair, rigorous peer review,
  broad scope, and wide readership – a perfect fit for your research every time.
  </p>
<p class="button-contain special">
    <a class="button button-default" href="/plosone/static/publish">
     Learn More
    </a>
    <a class="button-link" href="https://www.editorialmanager.com/pone/default.asp">
      Submit Now
    </a>
  </p>
<p>Click through the PLOS taxonomy to find articles in your field.</p>
<p>For more information about PLOS Subject Areas, click
          <a href="https://github.com/PLOS/plos-thesaurus/blob/master/README.md" target="_blank" title="Link opens in new window">here</a>.
        </p>
<p>Loading metrics</p>
<p class="license-short" id="licenseShort">Open Access</p>
<p class="peer-reviewed" id="peerReviewed">Peer-reviewed</p>
<p class="type-article" id=

dict_keys(['0.0', '1.0'])

In [142]:
(next(iter(article_dict.values())))

{'journal': 'PLOS One',
 'title': 'Impact of severe aortic stenosis on quality of life',
 'url': 'https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0287508',
 'text': '<h3 class="callout-headline">Submit Your Manuscript</h3>\n<p class="callout-content">\n  Discover a faster, simpler path to publishing in a high-quality journal. <em>PLOS ONE</em> promises fair, rigorous peer review,\n  broad scope, and wide readership – a perfect fit for your research every time.\n  </p>\n<p class="button-contain special">\n    <a class="button button-default" href="/plosone/static/publish">\n     Learn More\n    </a>\n    <a class="button-link" href="https://www.editorialmanager.com/pone/default.asp">\n      Submit Now\n    </a>\n  </p>\n<p>Click through the PLOS taxonomy to find articles in your field.</p>\n<p>For more information about PLOS Subject Areas, click\n          <a href="https://github.com/PLOS/plos-thesaurus/blob/master/README.md" target="_blank" title="Link opens in new wi

In [143]:
for article in article_dict:
    print(f"{article_dict[article]['journal']}: {article_dict[article]['text'][:100]}")

PLOS One: <h3 class="callout-headline">Submit Your Manuscript</h3>
<p class="callout-content">
  Discover a fa


KeyError: 'text'

## 2.65 Turns out it's an issue Annual Review of Medicine that gives response code 403

In [218]:
import scrapy
from scrapy.crawler import CrawlerRunner
from crochet import setup, wait_for
import re
from IPython import display
import time
from pprint import pprint
setup()

def trim_text(text, regex=None):
    if regex==None:
        regex = '.*<h2>Abstract</h2>.*(?:Introduction.*)?(<h2.*?>Introduction</h2>.*References)<.*' 
    try:
        processed = re.search(regex, text, re.DOTALL).group(1)
        html_display = display.HTML(processed)
    except: 
        print('Unable to parse article text')
        processed = '<Error parsing article text>' 
        html_display = processed
    return processed, html_display

def text_dict_from_web(article_dict, header=2, to_display=0,
        regex_str='.*<h\d>Abstract</h\d>.*(?:Introduction.*)?(<h\d.*?>Introduction</h\d>.*References)<.*'
        ):
    """
    Create a text dictionary from a dictionary containing web-scraped articles.

    Parameters:
        article_dict (dict): Values of each dictionary item are a dictionary representing the data from a 
            single article: 'url', 'text', and 'title'.

    Returns:
        text_dict: Dictionary where each item is a string of the text of an article, starting with the title.
    """
    journal = next(iter(article_dict.values()))['journal']
    print(f'Parsing {len(article_dict)} articles from {journal}')
    regex_str = regex_str.replace('\d', f'{header}')
    regex = rf'{regex_str}'
    print(f'Regex pattern: {regex}')
    text_dict = dict()
    display_dict = dict()
    if type(to_display) != list:
        to_display = [to_display] 
    for article_key in article_dict:
        trimmed_text, display = trim_text(article_dict[article_key]['text'], regex)
        text_dict[article_key] = f"{article_dict[article_key]['title']}\n\n{trimmed_text}"
        if article_key in to_display:
            display_dict[article_key] = display
    print(f'text_dict keys: {[key for key in text_dict.keys()]}')
    return text_dict, display_dict

class crawler_RSS1(scrapy.Spider):
    name = "crawler_RSS1"
    
    def start_requests(self):
        journals = {
            'PLOS One': 'https://journals.plos.org/plosone/feed/atom',
            # 'Annual Review of Medicine': 'https://www.annualreviews.org/action/showFeed?ui=45mu4&mi=3fndc3&ai=sm&jc=med&type=etoc&feed=atom' # response code 403
            }
        for index, journal in enumerate(journals):
            # article_dict[index] = dict()
            yield scrapy.Request(
                url=journals[journal], callback=self.parse_front, 
                cb_kwargs={'journal': journal, 'journal_index': index, 'article_dict': article_dict}
                )
    
    def parse_front(self, response, journal, journal_index, article_dict):
        response.selector.remove_namespaces() # This is needed for any Atom feeds
        # self.article_title = response.xpath('//entry/title/text()').getall()
        # article_url = response.css('entry > link[rel="alternate"]::attr(href)').getall()
        self.article_title = [response.xpath('//entry/title/text()').get()]
        article_url = [response.css('entry > link[rel="alternate"]::attr(href)').get()]
        for index, url in enumerate(article_url):
            # print(url)
            article_dict[f'{journal_index}.{index}'] = {
                'journal': journal,
                'title': self.article_title[index],
                'url': url
            }
            yield response.follow(
                url=url, callback=self.parse_pages, 
                cb_kwargs={'index': index, 'journal_index': journal_index, 'article_dict': article_dict})
                
    
    def parse_pages(self, response, index, journal_index, article_dict):
        # print(journal_index, index)
        # article_dict[f'{journal_index}.{index}']['test'] = 'test'
        text = response.xpath('//h2|//p|//h3|//h4').extract()
        article_dict[f'{journal_index}.{index}']['text'] = ''.join([line+'\n' for line in text])
        
@wait_for(10)
def run_spider():
    crawler = CrawlerRunner()
    d = crawler.crawl(crawler_RSS1)
    return d



iteration_id = 2.65
# main_dict = dict()
article_dict = dict()

run_spider()
main_dict[iteration_id] = article_dict
article_dict.keys()
# time.sleep(10)
# text_dict, display_dict = text_dict_from_web(article_dict, to_display=[0])

dict_keys(['0.0'])

In [206]:
print(article_dict['0.0'].keys())

dict_keys(['journal', 'title', 'url', 'text'])


In [210]:
next(iter(article_dict.values()))

{'journal': 'PLOS One',
 'title': 'Impact of severe aortic stenosis on quality of life',
 'url': 'https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0287508',
 'text': '<h3 class="callout-headline">Submit Your Manuscript</h3>\n<p class="callout-content">\n  Discover a faster, simpler path to publishing in a high-quality journal. <em>PLOS ONE</em> promises fair, rigorous peer review,\n  broad scope, and wide readership – a perfect fit for your research every time.\n  </p>\n<p class="button-contain special">\n    <a class="button button-default" href="/plosone/static/publish">\n     Learn More\n    </a>\n    <a class="button-link" href="https://www.editorialmanager.com/pone/default.asp">\n      Submit Now\n    </a>\n  </p>\n<p>Click through the PLOS taxonomy to find articles in your field.</p>\n<p>For more information about PLOS Subject Areas, click\n          <a href="https://github.com/PLOS/plos-thesaurus/blob/master/README.md" target="_blank" title="Link opens in new wi

In [222]:
text_dict, display_dict = text_dict_from_web(article_dict, to_display=['0.0'])
display_dict['0.0']

Parsing 1 articles from PLOS One
Regex pattern: .*<h2>Abstract</h2>.*(?:Introduction.*)?(<h2.*?>Introduction</h2>.*References)<.*
text_dict keys: ['0.0']


## 2.66 This works for multiple articles

In [266]:
import scrapy
from scrapy.crawler import CrawlerRunner
from crochet import setup, wait_for
import re
from IPython import display
import time
from pprint import pprint
setup()

def trim_text(text, regex=None):
    if regex==None:
        regex = '.*<h2>Abstract</h2>.*(?:Introduction.*)?(<h2.*?>Introduction</h2>.*References)<.*' 
    try:
        processed = re.search(regex, text, re.DOTALL).group(1)
        html_display = display.HTML(processed)
    except: 
        print('Unable to parse article text')
        processed = '<Error parsing article text>' 
        html_display = processed
    return processed, html_display

def text_dict_from_web(article_dict, header=2, to_display=0,
        regex_str='.*<h\d>Abstract</h\d>.*(?:Introduction.*)?(<h\d.*?>Introduction</h\d>.*References)<.*'
        ):
    """
    Create a text dictionary from a dictionary containing web-scraped articles.

    Parameters:
        article_dict (dict): Values of each dictionary item are a dictionary representing the data from a 
            single article: 'url', 'text', and 'title'.

    Returns:
        text_dict: Dictionary where each item is a string of the text of an article, starting with the title.
    """
    journal = next(iter(article_dict.values()))['journal']
    print(f'Parsing {len(article_dict)} articles from {journal}')
    regex_str = regex_str.replace('\d', f'{header}')
    regex = rf'{regex_str}'
    print(f'Regex pattern: {regex}')
    text_dict = dict()
    display_dict = dict()
    if type(to_display) != list:
        to_display = [to_display] 
    for article_key in article_dict:
        trimmed_text, display = trim_text(article_dict[article_key]['text'], regex)
        text_dict[article_key] = f"{article_dict[article_key]['title']}\n\n{trimmed_text}"
        if article_key in to_display:
            display_dict[article_key] = display
    print(f'text_dict keys: {[key for key in text_dict.keys()]}')
    return text_dict, display_dict

class crawler_RSS1(scrapy.Spider):
    name = "crawler_RSS1"
    
    def start_requests(self):
        journals = {
            'PLOS One': 'https://journals.plos.org/plosone/feed/atom',
            'BMJ Open': 'https://bmjopen.bmj.com/rss/current.xml'

            # 'Annual Review of Medicine': 'https://www.annualreviews.org/action/showFeed?ui=45mu4&mi=3fndc3&ai=sm&jc=med&type=etoc&feed=atom' # response code 403
            }
        for index, journal in enumerate(journals):
            # article_dict[index] = dict()
            yield scrapy.Request(
                url=journals[journal], callback=self.parse_front, 
                cb_kwargs={'journal': journal, 'journal_index': index, 'article_dict': article_dict}
                )
    
    def parse_front(self, response, journal, journal_index, article_dict):
        response.selector.remove_namespaces() # This is needed for any Atom feeds
        
        # self.article_title = response.xpath('//entry/title/text()').getall()
        # article_url = response.css('entry > link[rel="alternate"]::attr(href)').getall()
        self.article_title = [response.xpath('//entry/title/text()').get()]
        article_url = [response.css('entry > link[rel="alternate"]::attr(href)').get()]
        if article_url[0] == None:
            try:
                print('Extracting using method 2')
                self.article_title = [response.xpath('//item/title').get()]
                article_url = [response.css('item > link::text').get()]
                # self.article_title = [response.xpath('//item/title').getall()]
                # article_url = [response.css('item > link::text').getall()]
            except:
                print('error')
        for index, url in enumerate(article_url):
            print(url)
            key = float(f'{journal_index}.{index}')
            article_dict[key] = {
                'journal': journal,
                'title': self.article_title[index],
                'url': url
            }
            yield response.follow(
                url=url, callback=self.parse_pages, 
                cb_kwargs={'key': key, 'article_dict': article_dict})
                
    
    def parse_pages(self, response, key, article_dict):
        print(f'Journal #{key}')
        # article_dict[f'{journal_index}.{index}']['test'] = 'test'
        text = response.xpath('//h2|//p|//h3|//h4').extract()
        article_dict[key]['text'] = ''.join(['\n'+line for line in text])
        if key - int(key) == 0:
            print(f'\t{article_dict[key]["journal"]}')
            print(f'\tArticle attributes: {[key for key in article_dict[key].keys()]}')
        
@wait_for(10)
def run_spider():
    crawler = CrawlerRunner()
    d = crawler.crawl(crawler_RSS1)
    return d



iteration_id = 2.66
# main_dict = dict()
article_dict = dict()

run_spider()
main_dict[iteration_id] = article_dict
article_dict.keys()
# time.sleep(10)
# text_dict, display_dict = text_dict_from_web(article_dict, to_display=[0])

Extracting using method 2
http://bmjopen.bmj.com/cgi/content/short/13/6/e068887?rss=1
https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0287508
Journal #0.0
	PLOS One
	Article attributes: ['journal', 'title', 'url', 'text']
Journal #1.0
	BMJ Open
	Article attributes: ['journal', 'title', 'url', 'text']


dict_keys([1.0, 0.0])

## 2.7

In [270]:
import scrapy
from scrapy.crawler import CrawlerRunner
from crochet import setup, wait_for
import re
from IPython import display
import time
from pprint import pprint
setup()

def trim_text(text, regex=None):
    if regex==None:
        regex = '.*<h2>Abstract</h2>.*(?:Introduction.*)?(<h2.*?>Introduction</h2>.*References)<.*' 
    try:
        processed = re.search(regex, text, re.DOTALL).group(1)
        html_display = display.HTML(processed)
    except: 
        print('Unable to parse article text')
        processed = '<Error parsing article text>' 
        html_display = processed
    return processed, html_display

def text_dict_from_web(article_dict, header=2, to_display=0,
        regex_str='.*<h\d>Abstract</h\d>.*(?:Introduction.*)?(<h\d.*?>Introduction</h\d>.*References)<.*'
        ):
    """
    Create a text dictionary from a dictionary containing web-scraped articles.

    Parameters:
        article_dict (dict): Values of each dictionary item are a dictionary representing the data from a 
            single article: 'url', 'text', and 'title'.

    Returns:
        text_dict: Dictionary where each item is a string of the text of an article, starting with the title.
    """
    journal = next(iter(article_dict.values()))['journal']
    print(f'Parsing {len(article_dict)} articles from {journal}')
    regex_str = regex_str.replace('\d', f'{header}')
    regex = rf'{regex_str}'
    print(f'Regex pattern: {regex}')
    text_dict = dict()
    display_dict = dict()
    if type(to_display) != list:
        to_display = [to_display] 
    for article_key in article_dict:
        trimmed_text, display = trim_text(article_dict[article_key]['text'], regex)
        text_dict[article_key] = f"{article_dict[article_key]['title']}\n\n{trimmed_text}"
        if article_key in to_display:
            display_dict[article_key] = display
    print(f'text_dict keys: {[key for key in text_dict.keys()]}')
    return text_dict, display_dict

class crawler_RSS1(scrapy.Spider):
    name = "crawler_RSS1"
    
    def __init__(self, method='all'):
        self.method = method
    
    def start_requests(self):
        journals = {
            'PLOS One': 'https://journals.plos.org/plosone/feed/atom',
            'BMJ Open': 'https://bmjopen.bmj.com/rss/current.xml'

            # 'Annual Review of Medicine': 'https://www.annualreviews.org/action/showFeed?ui=45mu4&mi=3fndc3&ai=sm&jc=med&type=etoc&feed=atom' # response code 403
            }
        for index, journal in enumerate(journals):
            # article_dict[index] = dict()
            yield scrapy.Request(
                url=journals[journal], callback=self.parse_front, 
                cb_kwargs={'journal': journal, 'journal_index': index, 'article_dict': article_dict}
                )
    
    def parse_front(self, response, journal, journal_index, article_dict):
        response.selector.remove_namespaces() # This is needed for any Atom feeds
        if method == 'all':
            self.article_title = response.xpath('//entry/title/text()').getall()
            article_url = response.css('entry > link[rel="alternate"]::attr(href)').getall()
        else:
            self.article_title = [response.xpath('//entry/title/text()').get()]
            article_url = [response.css('entry > link[rel="alternate"]::attr(href)').get()]
        if article_url[0] == None:
            try:
                print('Extracting using method 2')
                if method == 'all':
                    self.article_title = response.xpath('//item/title').getall()
                    article_url = response.css('item > link::text').getall()
                else:
                    self.article_title = [response.xpath('//item/title').get()]
                    article_url = [response.css('item > link::text').get()]
            except:
                print('error')
        for index, url in enumerate(article_url):
            print(url)
            key = float(f'{journal_index}.{index}')
            article_dict[key] = {
                'journal': journal,
                'title': self.article_title[index],
                'url': url
            }
            yield response.follow(
                url=url, callback=self.parse_pages, 
                cb_kwargs={'key': key, 'article_dict': article_dict})
                
    
    def parse_pages(self, response, key, article_dict):
        print(f'Journal #{key}')
        # article_dict[f'{journal_index}.{index}']['test'] = 'test'
        text = response.xpath('//h2|//p|//h3|//h4').extract()
        article_dict[key]['text'] = ''.join(['\n'+line for line in text])
        if key - int(key) == 0:
            print(f'\t{article_dict[key]["journal"]}')
            print(f'\tArticle attributes: {[key for key in article_dict[key].keys()]}')
        
@wait_for(10)
def run_spider(method):
    crawler = CrawlerRunner()
    d = crawler.crawl(crawler_RSS1, method)
    return d



iteration_id = 2.7
# main_dict = dict()
article_dict = dict()
method = 'first'
run_spider(method)

main_dict[iteration_id] = article_dict
article_dict.keys()
# time.sleep(10)
# text_dict, display_dict = text_dict_from_web(article_dict, to_display=[0])

https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0287508
Extracting using method 2
http://bmjopen.bmj.com/cgi/content/short/13/6/e068887?rss=1
Journal #0.0
	PLOS One
	Article attributes: ['journal', 'title', 'url', 'text']
Journal #1.0
	BMJ Open
	Article attributes: ['journal', 'title', 'url', 'text']


dict_keys([0.0, 1.0])

## 2.71

In [309]:
import scrapy
from scrapy.crawler import CrawlerRunner
from crochet import setup, wait_for
import re
from IPython import display
import time
from pprint import pprint
setup()

def trim_text(text, regex=None):
    if regex==None:
        regex = '.*<h2>Abstract</h2>.*(?:Introduction.*)?(<h2.*?>Introduction</h2>.*References)<.*' 
    try:
        processed = re.search(regex, text, re.DOTALL).group(1)
        html_display = display.HTML(processed)
    except: 
        print('Unable to parse article text')
        processed = '<Error parsing article text>' 
        html_display = processed
    return processed, html_display

def text_dict_from_web(article_dict, header=2, to_display=0,
        regex_str='.*<h\d>Abstract</h\d>.*(?:Introduction.*)?(<h\d.*?>Introduction</h\d>.*References)<.*'
        ):
    """
    Create a text dictionary from a dictionary containing web-scraped articles.

    Parameters:
        article_dict (dict): Values of each dictionary item are a dictionary representing the data from a 
            single article: 'url', 'text', and 'title'.

    Returns:
        text_dict: Dictionary where each item is a string of the text of an article, starting with the title.
    """
    journal = next(iter(article_dict.values()))['journal']
    print(f'Parsing {len(article_dict)} articles from {journal}')
    regex_str = regex_str.replace('\d', f'{header}')
    regex = rf'{regex_str}'
    print(f'Regex pattern: {regex}')
    text_dict = dict()
    display_dict = dict()
    if type(to_display) != list:
        to_display = [to_display] 
    for article_key in article_dict:
        trimmed_text, display = trim_text(article_dict[article_key]['text'], regex)
        text_dict[article_key] = f"{article_dict[article_key]['title']}\n\n{trimmed_text}"
        if article_key in to_display:
            display_dict[article_key] = display
    print(f'text_dict keys: {[key for key in text_dict.keys()]}')
    return text_dict, display_dict

class crawler_RSS1(scrapy.Spider):
    name = "crawler_RSS1"
    
    def __init__(self, method='all'):
        self.method = method
    
    def start_requests(self):
        journals = {
            'PLOS One': 'https://journals.plos.org/plosone/feed/atom',
            'BMJ Open': 'https://bmjopen.bmj.com/rss/current.xml'

            # 'Annual Review of Medicine': 'https://www.annualreviews.org/action/showFeed?ui=45mu4&mi=3fndc3&ai=sm&jc=med&type=etoc&feed=atom' # response code 403
            }
        for index, journal in enumerate(journals):
            # article_dict[index] = dict()
            yield scrapy.Request(
                url=journals[journal], callback=self.parse_front, 
                cb_kwargs={'journal': journal, 'journal_index': index, 'article_dict': article_dict}
                )
    
    def parse_front(self, response, journal, journal_index, article_dict):
        response.selector.remove_namespaces() # This is needed for any Atom feeds
        print('Initiation')
        try:
            if self.method == 'all':
                self.article_title = response.xpath('//entry/title/text()').getall()
                article_url = response.css('entry > link[rel="alternate"]::attr(href)').getall()
                if article_url == []:
                    print('Extracting using method 2')
                    self.article_title = response.xpath('//item/title').getall()
                    article_url = response.css('item > link::text').getall()
            else:
                self.article_title = [response.xpath('//entry/title/text()').get()]
                article_url = [response.css('entry > link[rel="alternate"]::attr(href)').get()]
                if article_url[0] is None:
                    print('Extracting using method 2')
                    self.article_title = [response.xpath('//item/title').get()]
                    article_url = [response.css('item > link::text').get()]
        except:
            print('fail')
        
        for index, url in enumerate(article_url):
            # print(url)
            key = float(f'{journal_index}.{index}')
            article_dict[key] = {
                'journal': journal,
                'title': self.article_title[index],
                'url': url
            }
            yield response.follow(
                url=url, callback=self.parse_pages, 
                cb_kwargs={'key': key, 'article_dict': article_dict})
                
    
    def parse_pages(self, response, key, article_dict):
        # print(f'Journal #{key}')
        # article_dict[f'{journal_index}.{index}']['test'] = 'test'
        text = response.xpath('//h2|//p|//h3|//h4').extract()
        article_dict[key]['text'] = ''.join(['\n'+line for line in text])
        if key - int(key) == 0:
            print(f'\t{article_dict[key]["journal"]}')
            print(f'\tArticle attributes: {[key for key in article_dict[key].keys()]}')
        
@wait_for(15)
def run_spider(method):
    crawler = CrawlerRunner()
    d = crawler.crawl(crawler_RSS1, method)
    return d



iteration_id = 2.71
# main_dict = dict()
article_dict = dict()
method = 'first'
method = 'all'
run_spider(method)

main_dict[iteration_id] = article_dict
article_dict.keys()
# time.sleep(10)
# text_dict, display_dict = text_dict_from_web(article_dict, to_display=[0])

Initiation
Extracting using method 2
Initiation
	PLOS One
	Article attributes: ['journal', 'title', 'url', 'text']
	BMJ Open
	Article attributes: ['journal', 'title', 'url', 'text']


dict_keys([1.0, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 1.11, 1.12, 1.13, 1.14, 1.15, 1.16, 1.17, 1.18, 1.19, 1.21, 1.22, 1.23, 1.24, 1.25, 1.26, 0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.11, 1.27, 1.28, 1.29, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.21, 0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29])

In [310]:
article_dict[1.9]

{'journal': 'BMJ Open',
 'title': '<title>Persistent mortality and heart failure burden of anterior ST-segment elevation myocardial infarction following primary percutaneous coronary intervention: real-world evidence from the US Medicare Data Set</title>',
 'url': 'http://bmjopen.bmj.com/cgi/content/short/13/6/e071044?rss=1',
 'text': '\n<p class="logo-bmj-journals"><a href="http://journals.bmj.com/" title="BMJ Journals"><img src="/sites/default/themes/bmjj/img/logos/logo-bmj-journals.svg" title="BMJ Journals" alt="BMJ Journals"></a></p>\n<h3>Log in using your username and password</h3>\n<h2 class="pane-title"><span class="icon-search"></span></h2>\n<h2 class="pane-title"><span class="icon-menu"></span></h2>\n<h2 class="pane-title">Main menu</h2>\n<h3>Log in using your username and password</h3>\n<h2 class="element-invisible">You are here</h2>\n<p><span class="icon-article-text"></span>Article Text</p>\n<p><a class="article-pdf-download" href="/content/bmjopen/13/6/e071044.full.pdf" ta

# 3: Annual Review of Public Health

In [32]:
import scrapy
from scrapy.crawler import CrawlerRunner
from crochet import setup, wait_for
import re
from IPython import display
import time
from pprint import pprint
setup()

def trim_text(text, regex=None):
    if regex==None:
        regex = '.*<h2>Abstract</h2>.*(?:Introduction.*)?(<h2.*?>Introduction</h2>.*References)<.*' 
    try:
        processed = re.search(regex, text, re.DOTALL).group(1)
        html_display = display.HTML(processed)
    except: 
        print('Unable to parse article text')
        processed = '<Error parsing article text>' 
        html_display = processed
    return processed, html_display

def text_dict_from_web(article_dict, header=2, to_display=0,
        regex_str='.*<h\d>Abstract</h\d>.*(?:Introduction.*)?(<h\d.*?>Introduction</h\d>.*References)<.*'
        ):
    """
    Create a text dictionary from a dictionary containing web-scraped articles.

    Parameters:
        article_dict (dict): Values of each dictionary item are a dictionary representing the data from a 
            single article: 'url', 'text', and 'title'.

    Returns:
        text_dict: Dictionary where each item is a string of the text of an article, starting with the title.
    """
    journal = article_dict[0]['journal']
    print(f'Parsing {len(article_dict)} articles from {journal}')
    regex_str = regex_str.replace('\d', f'{header}')
    regex = rf'{regex_str}'
    print(f'Regex pattern: {regex}')
    text_dict = dict()
    display_dict = dict()
    if type(to_display) != list:
        to_display = [to_display] 
    for article_key in article_dict:
        trimmed_text, display = trim_text(article_dict[article_key]['text'], regex)
        text_dict[article_key] = f"{article_dict[article_key]['title']}\n\n{trimmed_text}"
        if article_key in to_display:
            display_dict[article_key] = display
    print(f'text_dict keys: {[key for key in text_dict.keys()]}')
    return text_dict, display_dict

class crawler_RSS1(scrapy.Spider):
    name = "crawler_RSS1"
    
    def start_requests(self):
        urls = ['https://www.annualreviews.org/pb-assets/journal-assets/rss/publhealth-rss-1548361333203.xml']
        for url in urls:
            yield scrapy.Request(url=url, callback=self.parse_front)
    
    def parse_front(self, response):
        article_url = [response.xpath('//item/title/text()').getall()]
        # self.article_title = response.xpath('//entry/title/text()').extract()
        # article_url = response.xpath('//entry/link[@rel="alternate"]/@href').extract()
        # article_url = response.css('entry > link[rel="alternate"][href]').extract()
        # self.article_title = response.css('entry > link[rel="alternate"]').extract()
        # article_url = response.css('entry > link[rel="alternate"][href]').extract()
        # self.article_title = response.css('link[rel="alternate"][title]').extract()
        # article_url = response.css('link[rel="alternate"][href]').extract()


        # article_url = response.xpath('entry > link::attr()"]/@href').extract()
        # self.article_title = [response.css('a.art-title > font::text').extract_first()]
        # article_url = [response.xpath('//a[@class="art-title"]/@href').extract_first()]
        for index, url in enumerate(article_url):
            print(url)
            # article_dict[index] = dict()
            # article_dict[index]['journal'] = 'BMJ Open'
            # article_dict[index]['title'] = self.article_title[index]
            # article_dict[index]['url'] = url
    #         yield response.follow(url=url, callback=self.parse_pages, cb_kwargs={'index': index})
    
    # def parse_pages(self, response, index):
    #     text = response.xpath('//h2|//p|//h3|//h4').extract()
    #     article_dict[index]['text'] = ''.join([line for line in text])
        
@wait_for(10)
def run_spider():
    crawler = CrawlerRunner()
    d = crawler.crawl(crawler_RSS1)
    return d



iteration_id = 3
# main_dict = dict()
article_dict = dict()

run_spider()
main_dict[iteration_id] = article_dict
article_dict
# time.sleep(10)
# text_dict, display_dict = text_dict_from_web(article_dict, to_display=[0])

['Making Health Research Matter: A Call to Increase Attention to External Validity', 'Causal Modeling in Environmental Health', 'Environmental Exposures and Depression: Biological Mechanisms and Epidemiological Evidence', 'Brain and Salivary Gland Tumors and Mobile Phone Use: Evaluating the Evidence from Various Epidemiological Study Designs', 'Innovations in Mixed Methods Evaluations', 'Earth Observation: Investigating Noncommunicable Diseases from Space', "Causes and Patterns of Dementia: An Update in the Era of Redefining Alzheimer's Disease", 'Realist Synthesis for Public Health: Building an Ontologically Deep Understanding of How Programs Work, For Whom, and In Which Contexts', 'Television News Coverage of Public Health Issues and Implications for Public Health Policy and Practice', 'Commentary: Causal Inference for Social Exposures', 'Policies of Exclusion: Implications for the Health of Immigrants and Their Children', 'The Economic Case for the Prevention of Mental Illness', 'Th

{}

# *End of Page*